In [1]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
from itertools import product

# --- 1. Pfad-Setup (identisch zu ET/EN/LGBM) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2. Repo-Imports (identisch zur ET-Logik) ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,
    EWMA_CORR_SPEC,
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)

# --- MODELL-IMPORT (TabPFN-Wrapper) ---
from src.models.TabPFN import ForecastModel


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code


In [2]:
# --- 3. Master-Schalter & Modellname (wie ET) ---
USE_DYNAMIC_FI_PIPELINE = False  # False => Full FE; True => Dynamic FI
SEED   = 42

if USE_DYNAMIC_FI_PIPELINE:
    MODEL_NAME = "tabpfn_dynamic_fi"
else:
    MODEL_NAME = "tabpfn"

outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")


Modell tabpfn wird getunt.


In [3]:
# --- 4. Daten laden (identisch zu ET) ---
FI_PATH = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"

y = load_target()           # ΔIP (DatetimeIndex)
X_ifo = load_ifo_features() # ifo features (Full FE)

if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_PATH)
        rolling_imp   = load_rolling_importance(base_dir=FI_PATH)
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi, X_full_lagged, rolling_imp = y.loc[idx_fi], X_full_lagged.loc[idx_fi], rolling_imp.loc[idx_fi]
        print("Dynamic-FI Daten geladen. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER: Dynamic-FI Artefakte fehlen:", e)
        print("Bitte erst feature_importance.ipynb (entsprechende Jobs) laufen lassen.")
        raise
else:
    X_full_lagged, rolling_imp = None, None
    idx = y.index.intersection(X_ifo.index)
    y, X_ifo = y.loc[idx], X_ifo.loc[idx]
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)


INFO in load_ifo_features: Renaming columns to ensure validity.
Full-FE Daten geladen. Shapes: (407, 2160) (407,)


In [4]:
# --- 5. Base-Config & Policy (wie Thesis, identisch zu ET) ---
def base_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg()

# --- 6. Corr-Helper (identisch zu ET) ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    elif kind == "ewm":
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")


In [5]:
# --- 7. TUNING-GRID (analog zur ET-Struktur, aber TabPFN hat kaum Modell-HPs) ---

# WICHTIG: Die Device-Wahl soll NICHT über Grid getuned werden (konstant halten).
USE_GPU_DEFAULT = False     # <— ändere hier, wenn du GPU nutzen willst
FORCE_DEVICE = None         # 'cpu' | 'cuda' | 'mps' oder None (Auto basierend auf USE_GPU_DEFAULT)

if USE_DYNAMIC_FI_PIPELINE:
    print("Erstelle HP-Grid für 'Dynamic FI' ...")

    N_FEATURES_TO_USE = 20
    weighting_options = [
        {"sample_weight_decay": None},
        {"sample_weight_decay": 0.98},
    ]

    def build_model_grid_dynamic_fi():
        grid = []
        base = {
            "n_features_to_use": N_FEATURES_TO_USE,
            "seed": SEED,
            "use_gpu": USE_GPU_DEFAULT,
            "device": FORCE_DEVICE,   # None => Auto im Wrapper
        }
        for w in weighting_options:
            hp = dict(base)
            hp.update(w)
            grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    print("Erstelle HP-Grid für 'Full FE' ...")

    # A) FE/DR-Listen (wie ET, reduziert)
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
        ("ewm",       make_corr_spec("ewm")),
    ]
    lag_candidates_list   = [(1, 2, 3, 6, 12)]
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100, 300]
    redundancy_param_list = [0.90]
    dr_options_list       = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_var_target": 0.95, "pca_kmax": 50},
        {"dr_method": "pls", "pls_components": 8},
    ]

    # B) Target Blocks & Weighting
    target_block_options = [None, ["AR1"], ["Chronos"], ["TSFresh"]]
    weighting_options    = [{"sample_weight_decay": None}]

    # C) Grid zusammensetzen (TabPFN hat keine weiteren sinnvollen Modell-HPs hier)
    def build_model_grid_full_fe():
        hp_grid = []
        fe_lists = [
            lag_candidates_list,
            top_k_lags_list,
            use_rm3_list,
            k1_topk_list,
            redundancy_param_list,
            dr_options_list,
        ]
        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                if k1 == 100 and dr_opt["dr_method"] != "none":
                    continue
                base = {
                    "lag_candidates": lags,
                    "top_k_lags_per_feature": k_lags,
                    "use_rm3": rm3,
                    "k1_topk": k1,
                    "redundancy_param": red,
                    **dr_opt,
                    "corr_tag": corr_tag,
                    "corr_spec": corr_spec,
                    # TabPFN-spezifisch (konstant, kein Tuning):
                    "seed": SEED,
                    "use_gpu": USE_GPU_DEFAULT,
                    "device": FORCE_DEVICE,
                }
                for block_set in target_block_options:
                    for w in weighting_options:
                        hp = {**base, "target_block_set": block_set, **w}
                        hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("Optimierte HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")


Erstelle HP-Grid für 'Full FE' ...
Optimierte HP-Kombinationen: 32
Erstes HP-Set: {'lag_candidates': (1, 2, 3, 6, 12), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'seed': 42, 'use_gpu': False, 'device': None, 'target_block_set': None, 'sample_weight_decay': None}


In [6]:
# --- 8. Stage A/B Lauf (identisch zur ET-Logik, inkl. Dynamic-FI Pfad) ---
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,  # Platzhalter, Dynamic-FI nutzt X_full_lagged + rolling_imp
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(10, len(model_grid)),
            min_survivors_per_block=2,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(10, len(model_grid)),
            min_survivors_per_block=2,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")


[Stage A] Using FULL FE pipeline (Gleis 1 & 2).
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=32
  - Config 1/32
    · Month 5/20 processed | running...RMSE=1.6724
    · Month 10/20 processed | running...RMSE=1.3424
    · Month 15/20 processed | running...RMSE=1.2585
    · Month 20/20 processed | running...RMSE=1.1273
  - Config 2/32
    · Month 5/20 processed | running...RMSE=1.6724
    · Month 10/20 processed | running...RMSE=1.3424
    · Month 15/20 processed | running...RMSE=1.2585
    · Month 20/20 processed | running...RMSE=1.1273
  - Config 3/32


KeyboardInterrupt: 